In [201]:
import json, pprint, os

In [298]:
EVENT_NAME       = "2016 Hurricane Matthew: Brunswick: Contextual"
html_output_file = "/home/anderstj/tweetsonamap.com/docs/evacuation/events/brunswick-contextual.html"
input_directory  = "/data/chime/geo2/NEW/BRUNSWICK/"

ROOT             = "https://s3-us-west-1.amazonaws.com/movement-derivation/cdn/NEW/BRUNSWICK/"
USER_URL = "../index.html?user={0}&root="+ROOT

## Go run this code:

```
aws --profile=epic s3 cp --recursive --acl=public-read /data/chime/geo2/NEW/contextual-stream-test s3://movement-derivation/cdn/NEW/contextual-stream-test
```

In [299]:
def load_user_metas(input_directory):
    users = [x for x in os.listdir(input_directory) if x.endswith('-meta.geojson') ]
    print("Found {0} metadata files".format(len(users)))
    return users

In [300]:
users = load_user_metas(input_directory)
home_locations = {"type":"FeatureCollection","features":[]}

Found 72 metadata files


In [301]:
def rankUser(u):
    #Read the metadata file and do something
    score = 0;
    clustered_tweets = 0;
    
    this_user = json.load(open(input_directory + u))
    
    handle = u[0:-13]
    
    home_loc = False;
    
    clusters = [x for x in this_user['features'] if x['properties'].get('clusterCenter',False)]

    for cluster in clusters:
        if cluster['properties'].get('likelyHome',False):
            score += 1;
            home_loc = cluster;
        elif  cluster['properties'].get('likelyWork',False):
            score += 1;
        clustered_tweets += cluster['properties']['tweetCount']
        
    if home_loc:
        home_loc['properties'] = {
            'handle': handle,
            'link' : USER_URL.format(handle),
            'totalClusteredTweets': clustered_tweets,
            'totalClusters': len(clusters)
        }
        home_locations['features'].append(home_loc)
    
    return (score, len(clusters), clustered_tweets, handle)

In [302]:
print("Score | #Clusters | #Tweets | User")
for u in users[:10]:
    score, clusterCount, tweetCount, handle = rankUser(u)
    print("{0}        {1}        {2}           {3}".format(score, clusterCount, tweetCount, handle))

Score | #Clusters | #Tweets | User
0        0        0           bethel1415
2        5        19           mys41tic
0        0        0           romantsunami_67
0        0        0           gilliardfarms
0        0        0           k_engram
0        0        0           aquafae
0        0        0           frchrisortega
0        1        7           nothinshockin
0        1        3           tylermay_23
0        0        0           mapintl


### Write out HTML

In [303]:
def makeTableRow(handle, tweetCount, clusterCount):
    string = """<tr>
    <td><a target="_blank" class="link" href="{3}">{0}</a></td>
    <td>{1}</td>
    <td>{2}</td>
    </tr>""".format(handle, tweetCount, clusterCount, USER_URL.format(handle,ROOT))
    return [string, tweetCount]

In [304]:
twos = []
ones = []
zeroes = []
for u in users:
    try:
        score, clusterCount, tweetCount, handle = rankUser(u)

        if score==2:
            twos.append(makeTableRow(handle, tweetCount, clusterCount))

        if score==1:
            ones.append(makeTableRow(handle, tweetCount, clusterCount))

        if score==0:
            zeroes.append(makeTableRow(handle, tweetCount, clusterCount))
    except:
        print("error on "+u)
print(len(twos), len(ones), len(zeroes))

10 7 55


In [305]:
home_locations['features'] = home_locations['features']

CONTENT = ""
CONTENT += "<h4 class='txt-h4'>Users with calculated home & work location</h4>"
CONTENT += "<table>\n<thead><tr><th>User</th><th>Clustered Tweets</th><th># Clusters</th><tr></thead>\n<tbody>"
for row in sorted(twos, key=lambda x: -x[1]):
    CONTENT += row[0]+"\n"
CONTENT += "</tbody></table>"

CONTENT += "<h4 class='txt-h4'>Users with home or work location determined</h4>"
CONTENT += "<table>\n<thead><tr><th>User</th><th>Clustered Tweets</th><th># Clusters</th><tr></thead>\n<tbody>"
for row in sorted(ones, key=lambda x: -x[1]):
    CONTENT += row[0]+"\n"
CONTENT += "</tbody></table>"
 
CONTENT += "<h4 class='txt-h4'>Users with neither clustered home or work location</h4>"
CONTENT += "<table>\n<thead><tr><th>User</th><th>Clustered Tweets</th><th># Clusters</th><tr></thead>\n<tbody>"
for row in sorted(zeroes, key=lambda x: -x[1]):
    CONTENT += row[0]+"\n"
CONTENT += "</tbody></table>"
CONTENT += "<script type='text/javascript'>"
CONTENT += "var homeLocations = " + json.dumps(home_locations)
CONTENT += "</script>"

In [306]:
TEMPLATE = open('/home/anderstj/tweetsonamap.com/_templates/event_page.html_template','r').read()

In [307]:
with open(html_output_file,'w') as outFile:
    outFile.write(TEMPLATE.format(EVENT_NAME, EVENT_NAME, CONTENT))